In [29]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### Get list of episode URLs ###

In [18]:
def getEpisodeLinks(URL):   
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    out=[]
    for a in soup.find_all('a',href=True):
        out.append(a['href'])
    return out[64:84]

def getIdentifiers(list):
        ids=[x[-8:] for x in list]
        return set(ids)
    

In [19]:
urlPages = [f'https://www.bbc.co.uk/programmes/b006qnmr/episodes/player?page={x}' for x in range(201)]
links=[]
for x in range(1,201): # make this 201 next time and get your links
    URL= urlPages[x]
    epLinks= getEpisodeLinks(URL)
    ids= getIdentifiers(epLinks)
    listIDs= list(ids)
    finalURLs=[]
    for y in listIDs:
        finalURLs.append(f"https://www.bbc.co.uk/programmes/{y}")
    links.append(finalURLs)

In [23]:
final_list= [item for sublist in links for item in sublist]
final_list= np.unique(final_list)

### Get synopsis and castaway ###

In [25]:
def getSynopses(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        content= soup.find('div', class_="synopsis-toggle__long").text
        castaway= soup.find('h1').text
        return [content, castaway,x]
    except: pass


In [26]:
out=[]
for x in final_list:
    out.append(getSynopses(x))

In [46]:
new_data=list(filter(lambda item: item is not None, out))
df= pd.DataFrame(new_data,columns=["synopsis","castaway","url"])
df.to_csv('synopsis_data.csv')